## Imports

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize 
import itertools


## Preprocessing

In [72]:
# Load the data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [73]:
processed_df = train_df.copy()
processed_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The first column represents the digit that the pixels represent and the columns pixel0:pixel783 are the pixels in a 28 by 28 image.

In [74]:
processed_df.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [75]:
processed_df.dtypes

label       int64
pixel0      int64
pixel1      int64
pixel2      int64
pixel3      int64
pixel4      int64
pixel5      int64
pixel6      int64
pixel7      int64
pixel8      int64
pixel9      int64
pixel10     int64
pixel11     int64
pixel12     int64
pixel13     int64
pixel14     int64
pixel15     int64
pixel16     int64
pixel17     int64
pixel18     int64
pixel19     int64
pixel20     int64
pixel21     int64
pixel22     int64
pixel23     int64
pixel24     int64
pixel25     int64
pixel26     int64
pixel27     int64
pixel28     int64
            ...  
pixel754    int64
pixel755    int64
pixel756    int64
pixel757    int64
pixel758    int64
pixel759    int64
pixel760    int64
pixel761    int64
pixel762    int64
pixel763    int64
pixel764    int64
pixel765    int64
pixel766    int64
pixel767    int64
pixel768    int64
pixel769    int64
pixel770    int64
pixel771    int64
pixel772    int64
pixel773    int64
pixel774    int64
pixel775    int64
pixel776    int64
pixel777    int64
pixel778  

## Exploratory Data Analysis

There seems to be an even number of each of the digits from 0 to 9 in the set.

In [76]:
y_train = train_df["label"]

# Drop 'label' column
x_train = train_df #.drop(["label"], 1) 

# free some space
# del train_df

x_train.groupby("label").count()["pixel0"]

label
0    4132
1    4684
2    4177
3    4351
4    4072
5    3795
6    4137
7    4401
8    4063
9    4188
Name: pixel0, dtype: int64

The data represents a the pixel value on a scale of 0 to 255.

In [77]:
x_train.max()

label         9
pixel0        0
pixel1        0
pixel2        0
pixel3        0
pixel4        0
pixel5        0
pixel6        0
pixel7        0
pixel8        0
pixel9        0
pixel10       0
pixel11       0
pixel12     116
pixel13     254
pixel14     216
pixel15       9
pixel16       0
pixel17       0
pixel18       0
pixel19       0
pixel20       0
pixel21       0
pixel22       0
pixel23       0
pixel24       0
pixel25       0
pixel26       0
pixel27       0
pixel28       0
           ... 
pixel754      0
pixel755      0
pixel756      0
pixel757      0
pixel758      0
pixel759      0
pixel760      0
pixel761    177
pixel762    231
pixel763    253
pixel764    254
pixel765    254
pixel766    255
pixel767    255
pixel768    255
pixel769    255
pixel770    255
pixel771    255
pixel772    255
pixel773    255
pixel774    254
pixel775    254
pixel776    253
pixel777    253
pixel778    254
pixel779     62
pixel780      0
pixel781      0
pixel782      0
pixel783      0
Length: 785, dtype: int6

## Normalization

Our Keras Convolutional Neural Network will perform better when the pixel values are normalized to avoid poor image quality (like contrast due to glare).

In [106]:
x_train = x_train / 255.0
test = test / 255.0
x_train

array([[3.60335571e-06, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.60335571e-06, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.52234900e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.16201343e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.24302014e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

## Reshape

In [98]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
x_train = x_train.values.reshape(-1,28,28,1)
test_df = test_df.values.reshape(-1,28,28,1)

g = plt.imshow(x_train[0][:,:,0])

AttributeError: 'numpy.ndarray' object has no attribute 'values'